In [105]:
# import packages
from collections import deque
import pandas as pd
import numpy as np
import random
import copy

In [106]:
## Classical DA! 
def find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy):
    
    #########################################
    #NOTE: When the number of boys are discrete with gaps
    #      turned into [1,2,3,... ] continuous number
    #########################################
    boy_index_dict = {}
    for i in range(len(boys)):
        boy_index_dict[boys[i]] = i+1
    girl_index_dict = {}
    for i in range(len(girls)):
        girl_index_dict[girls[i]] = i+1
    # old_boys = boys
    # old_girls = girls
    new_boys = list(range(1,len(boys)+1))
    new_girls = list(range(1,len(girls)+1))
    new_sort_boy_to_girl = []
    for items in sort_boy_to_girl:
        tmp = []
        for item in items:
            tmp.append(girl_index_dict[item])
        new_sort_boy_to_girl.append(tmp)
    new_sort_girl_to_boy = []
    for items in sort_girl_to_boy:
        tmp = []
        for item in items:
            tmp.append(boy_index_dict[item])
        new_sort_girl_to_boy.append(tmp)
    ###########################################################
    # print(girl_index_dict)
    # current choice
    # build dict：key: n-th boy，value: current choice of girl，current_boys = {boys[0]:None, boys[1]:None, boys[2]:None, boys[3]:None ... }
    current_boys = dict(zip(new_boys, [None]*len(new_boys)))
    # build dict：key: n-th girl，value: current choice of boy，current_girls = {girls[0]:None, girls[1]:None, girls[2]:None, boys[3]:None ... }
    current_girls = dict(zip(new_girls, [None]*len(new_girls)))
    # sumboys= sum(current_girls)
    count = len(boys)
 
 
    # build dict: next girl to choose for boys
    next_select = dict(zip(new_boys, [None]*len(new_boys)))
    # build sequence for every boy
    for i in range(count):
        # propose to the first girl
        temp = [new_girls[m-1] for m in new_sort_boy_to_girl[i]]
        # next time propose to the second
        next_select[new_boys[i]] = deque(temp)
 
 
    # girl choose boy dict
    sort_girl = dict(zip(new_girls, [None]*len(new_boys)))
    for i in range(count):
        # girl rank boy
        temp = [[new_boys[m-1], 19-ind] for ind, m in enumerate(new_sort_girl_to_boy[i])]
        name = []
        match = []
        # for every girl
        for t in temp:
            # number of matched boy
            name.append(t[0])
            # rank of matched boy
            match.append(t[1])
        # get the ranking of 
        sort_girl[new_girls[i]] = dict(zip(name, match))
 
 
    while None in current_boys.values():
        for i in range(count):
            # i-th boy proposes
            bid = new_boys[i]
            if current_boys[bid]:
                # skip if matched
                continue
            else:
                # choose the next on preference list
                select = next_select[bid][0]
                # if girl is unmatched
                if current_girls[select] == None:
                    # match
                    current_boys[bid] = select
                    current_girls[select] = bid
                    next_select[bid].popleft()
                else:
                    # compare propose boy to the current match
                    # if the current match is preferred to the propose boy
                    # keep status quo 
                    if sort_girl[select][current_girls[select]] > sort_girl[select][bid]:
                        next_select[bid].popleft()
                    # if the reverse
                    # propose boy is preferred to the current match
                    # unmatch
                    # the girl match with propose boy
                    # current match can't propose again
                    else:
                        current_boys[current_girls[select]] = None
                        current_boys[bid] = select
                        current_girls[select] = bid
                        next_select[bid].popleft()
    #NOTE:return to original index
    return_boys = dict()
    for key,value in current_boys.items():
        return_boys[boys[key-1]] = girls[value-1]
    return return_boys

In [107]:
#reindex the irregular cases!
def tr_numbers(source_match_dict, source_sort_boy_to_girl, source_sort_girl_to_boy):
    # if the number or the preference is irregular
    # boy_dict = {}
    tr_boy_dict = {}
    # girl_dict = {}
    tr_girl_dict = {}
    count = 1
    for i in range(len(source_sort_boy_to_girl[0])):
        # girl_dict[i+1] = source_sort_boy_to_girl[0][i]
        tr_girl_dict[source_sort_boy_to_girl[0][i]] = i+1
        # boy_dict[i+1] = source_sort_girl_to_boy[0][i]
    for key in source_match_dict.keys():
        tr_boy_dict[key] = count
        count += 1

    match_dict = {}
    sort_boy_to_girl = []
    sort_girl_to_boy = []
    for key,value in source_match_dict.items():
        match_dict[tr_boy_dict[key]] = tr_girl_dict[value] if value!=0 else 0
    for sort_list in source_sort_boy_to_girl:
        sort_boy_to_girl.append([tr_girl_dict[x] for x in sort_list]) 
    for sort_list in source_sort_girl_to_boy:
        sort_girl_to_boy.append([tr_boy_dict[x] for x in sort_list]) 

    return match_dict, sort_boy_to_girl, sort_girl_to_boy

In [108]:
#count blocking pairs
def count_blocking_pairs(match_dict, sort_boy_to_girl, sort_girl_to_boy):

    tr_match_dict = {i:0 for i in range(len(match_dict))}
    for key,value in match_dict.items():
        if value:
            tr_match_dict[value]=key

    count = 0
    all_match = 0
    for key,value in match_dict.items():
        # if not matched
        if value==0: 
            continue
        else:
            all_match += 1
            miss_match = False
            boy_to_girl_index = sort_boy_to_girl[key-1].index(value) #目前匹配的女生的排序
            if boy_to_girl_index!=0: 
                #if the ranking is not the highest
                # walk through all girls with higher preference
                for i in range(boy_to_girl_index):
                    girl_index = sort_boy_to_girl[key-1][i]  # current girl
                    girl_current_boy_index = tr_match_dict[girl_index] # current boy of current girl
                    # if the girl is not matched
                    if girl_current_boy_index == 0: 
                        miss_match = True
                        break
                    # if the girl is on a worse match
                    elif sort_girl_to_boy[girl_index-1].index(key)<sort_girl_to_boy[girl_index-1].index(girl_current_boy_index):
                        miss_match = True
                        break
                if miss_match:
                    continue
            
            girl_to_boy_index = sort_girl_to_boy[value-1].index(key)
            if girl_to_boy_index!=0: 
                # not highest
                # walk through all boys that rank higher on preference
                for i in range(girl_to_boy_index):
                    boy_index = sort_girl_to_boy[value-1][i] # current boy
                    boy_current_girl_index = match_dict[boy_index] # current girl of current girl
                    # if the boy is not matched
                    if boy_current_girl_index == 0: 
                        miss_match = True
                        break
                    # if the boy is on a worse match
                    elif sort_boy_to_girl[boy_index-1].index(value)<sort_boy_to_girl[boy_index-1].index(boy_current_girl_index):
                        miss_match = True
                        break
                if miss_match:
                    continue
            count += 1
    return count, all_match

In [110]:
# Standard matching until the end
# initialization
size=20
boys = list(range(1,size+1))
girls = list(range(1,size+1))
# mr_girl_sum: sum of boy rank for girl
# mr_boy_sum: sum of girl rank for boy
mr_girl_sum=0
mr_boy_sum=0
# amr_girl: average rank of matched boy rank for girl (mr_girl_sum/rep)
# amr_boy: average rank of matched girl rank for girl (mr_boy_sum/rep)
amr_girl=0
amr_boy=0
amr_boy_total=0
amr_girl_total=0
fe_repavg=0
# repeat
# rep: repeated time
# generate 1-20 random numbers for boy's preference of girls as a random matrix, 
sort_boy_to_girl= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
# print(sort_boy_to_girl)
# print boy's preference of girls as a dataframe 
dfboys = pd.DataFrame(pd.Series(sort_boy_to_girl),columns={'boy_preferece'})
# reindex: start from 1 
dfboys.index=dfboys.index+1
dfboys=dfboys.reset_index().rename(columns = {'index':'boy'})
# generate 1-20 random numbers for girl's preference of boys as a random matrix, 
sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
# print(sort_girl_to_boy)
# print  girl's preference of boys as a dataframe 
dfgirls = pd.DataFrame(pd.Series(sort_girl_to_boy),columns={'girl_preference'})
# reindex: start from 1 
dfgirls.index=dfgirls.index+1
dfgirls=dfgirls.reset_index().rename(columns = {'index':'girl'})
# dfgirls
# print the result of the first matching
result = find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy)
df = pd.DataFrame(pd.Series(result), columns=['girl_match'])
df=df.reset_index().rename(columns = {'index':'boy_match'})
#NOTE:test
new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy = tr_numbers(result,sort_boy_to_girl, sort_girl_to_boy)
blocking_num,all_match_num = count_blocking_pairs(new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy)
df
# count once
# calculate average ranking of girl for boy on boys' preference among all reps (mr_boy_sum/rep)
girl_match=new_match_dict
print(girl_match)
mr_boy_sum=0
mr_girl_sum=0
for key,value in girl_match.items():
    if value:
        mr_boy=new_sort_boy_to_girl[key-1].index(value)
        mr_girl=new_sort_girl_to_boy[value-1].index(key)
        mr_boy_sum += mr_boy+1
        mr_girl_sum += mr_girl+1

amr_boy=mr_boy_sum/all_match_num
print("amr_boy=",amr_boy)

# calculate average ranking for boys of girl on girls' preference among all reps (mr_boy_sum/rep)
amr_girl=mr_girl_sum/all_match_num
print("amr_girl = ",amr_girl)
print("amr_total = ",amr_boy/2+amr_girl/2)
print("fe = ",amr_boy/2-amr_girl/2)
print("number of matched pair = ", all_match_num)
print("number of blocking pairs = ", all_match_num)

{1: 1, 2: 9, 3: 13, 4: 16, 5: 14, 6: 20, 7: 7, 8: 5, 9: 4, 10: 15, 11: 19, 12: 18, 13: 12, 14: 17, 15: 2, 16: 8, 17: 3, 18: 6, 19: 10, 20: 11}
amr_boy= 3.45
amr_girl =  11.15
amr_total =  7.300000000000001
fe =  -3.85
number of matched pair =  20
number of blocking pairs =  20


In [111]:
# REPEATED Standard matching until the end
# replicate for rep times
rep = 10 #add
for i in range(1,rep):
    # generate random preference matrix
    # boys rank girls
    sort_boy_to_girl= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
    dfboys = pd.DataFrame(pd.Series(sort_boy_to_girl),columns={'boy_preferece'})
    dfboys.index=dfboys.index+1
    dfboys=dfboys.reset_index().rename(columns = {'index':'boy'})        
    sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
    dfgirls = pd.DataFrame(pd.Series(sort_girl_to_boy),columns={'girl_preference'})
    dfgirls.index=dfgirls.index+1
    dfgirls=dfgirls.reset_index().rename(columns = {'index':'girl'})
    girl_match=pd.Series(find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy))
    result = find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy)
    df = pd.DataFrame(pd.Series(result), columns=['girl_match'])
    df=df.reset_index().rename(columns = {'index':'boy_match'})
    #NOTE:test
    new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy = tr_numbers(result,sort_boy_to_girl, sort_girl_to_boy)
    blocking_num,all_match_num = count_blocking_pairs(new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy)
    df
    girl_match=new_match_dict
    print(girl_match)
    mr_boy_sum=0
    mr_girl_sum=0
    for key,value in girl_match.items():
        if value:
            mr_boy=new_sort_boy_to_girl[key-1].index(value)
            mr_girl=new_sort_girl_to_boy[value-1].index(key)
            mr_boy_sum += mr_boy+1
            mr_girl_sum += mr_girl+1
    amr_boy=mr_boy_sum/all_match_num
    amr_girl=mr_girl_sum/all_match_num
    print("amr_boy=",amr_boy)
    print("amr_girl=",amr_girl)
    print("amr_total=",amr_boy/2+amr_girl/2)
    print("fe=",amr_boy/2-amr_girl/2)
    print("number of matched pair = ", all_match_num)
    print("number of blocking pairs = ", all_match_num)
    amr_boy_total+=amr_boy
    amr_girl_total+=amr_girl
    fe_repavg+=amr_boy/2-amr_girl/2
    i+=1
    
print("amr_boy_repavg=",amr_boy_total/rep)
print("amr_girl_repavg=",amr_girl_total/rep)
print("amr_total_repavg=",amr_boy_total/2/rep+amr_girl_total/2/rep)
print("fe_repavg=",fe_repavg/rep)

{1: 7, 2: 5, 3: 16, 4: 19, 5: 17, 6: 3, 7: 20, 8: 13, 9: 11, 10: 10, 11: 14, 12: 6, 13: 12, 14: 9, 15: 2, 16: 4, 17: 8, 18: 1, 19: 18, 20: 15}
amr_boy= 4.35
amr_girl= 10.05
amr_total= 7.2
fe= -2.8500000000000005
number of matched pair =  20
number of blocking pairs =  20
{1: 4, 2: 7, 3: 1, 4: 13, 5: 18, 6: 11, 7: 19, 8: 20, 9: 10, 10: 2, 11: 16, 12: 15, 13: 14, 14: 6, 15: 5, 16: 12, 17: 3, 18: 9, 19: 17, 20: 8}
amr_boy= 3.0
amr_girl= 7.85
amr_total= 5.425
fe= -2.425
number of matched pair =  20
number of blocking pairs =  20
{1: 1, 2: 13, 3: 14, 4: 19, 5: 18, 6: 6, 7: 7, 8: 5, 9: 20, 10: 3, 11: 2, 12: 10, 13: 15, 14: 9, 15: 12, 16: 4, 17: 8, 18: 17, 19: 16, 20: 11}
amr_boy= 2.55
amr_girl= 11.1
amr_total= 6.824999999999999
fe= -4.275
number of matched pair =  20
number of blocking pairs =  20
{1: 2, 2: 12, 3: 13, 4: 8, 5: 9, 6: 14, 7: 3, 8: 15, 9: 17, 10: 7, 11: 5, 12: 16, 13: 4, 14: 20, 15: 6, 16: 11, 17: 18, 18: 10, 19: 19, 20: 1}
amr_boy= 3.75
amr_girl= 11.95
amr_total= 7.85
fe= -4.1

In [112]:
# random divorce rate
prob=0.6
size=all_match_num

# select 20*0.6=12 boys as divorced boys
crossover= random.sample(range(1,size+1),12)

In [113]:
# define a function: delete list from list
def diff_of_two_list(lst1, lst2): 
    diff_list = []
    for item in lst1:
        if item not in lst2:
            diff_list.append(item) 
    return diff_list

In [114]:
# divorced boys and divorced girls 
df2=df[df["boy_match"].isin(crossover)]
# df2
# divorced boys list
boy_divorce=df2['boy_match'].values.tolist()
boy_divorce.sort()
# divorced girls list
girl_divorce=df2['girl_match'].values.tolist()
girl_divorce.sort()

In [115]:
# not divorced boys and girls (to be deleted from the population, and get the divorced sample)
girl_not_divorce=diff_of_two_list(girls,girl_divorce)
boy_not_divorce=diff_of_two_list(boys,boy_divorce)
# girl_not_divorce

In [116]:
# divorced boys rank divorced girls: preference
if __name__ == '__main__':
    sort_boy_to_girl_divorce = []
    # for i in range(20)
    for i in boy_divorce:
        # preference of girls for boys, MINUS undivorced girls
        item=diff_of_two_list(sort_boy_to_girl[i-1],girl_not_divorce)
        sort_boy_to_girl_divorce.append(item) 
# sort_boy_to_girl_divorce

#  divorced girls rank divorced boys: preference
if __name__ == '__main__':
    sort_girl_to_boy_divorce = []
    # for i in range(20):#NOTE：same
    for i in girl_divorce:
        # preference of boys for girls, MINUS undivorced boys
        item=diff_of_two_list(sort_girl_to_boy[i-1],boy_not_divorce)
        sort_girl_to_boy_divorce.append(item) 
# sort_girl_to_boy_divorce

In [118]:
# Divorced matching until the end
## initialization
size=20
boys = list(range(1,size+1))
girls = list(range(1,size+1))
# mr_girl_sum: sum of rankings of matched boy for girls
# mr_boy_sum: sum of rankings of matched girl for boys
mr_girl_sum=0
mr_boy_sum=0
# amr_girl: average of rankings of matched boy for girls (mr_girl_sum/rep)
# amr_boy:  average of rankings of matched girl for boys (mr_boy_sum/rep)
amr_girl=0
amr_boy=0
amr_boy_total=0
amr_girl_total=0
fe_repavg=0
blocking_pairs=0
matched_pairs=0


result = find_free_partner(boy_divorce, girl_divorce, sort_boy_to_girl_divorce, sort_girl_to_boy_divorce) 
new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy = tr_numbers(result,sort_boy_to_girl_divorce, sort_girl_to_boy_divorce)
blocking_num,all_match_num = count_blocking_pairs(new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy)
divorce_match=pd.DataFrame(list(divorce_match.items()),columns=['boy', 'girl'])
girl_match=new_match_dict
for key,value in girl_match.items():
    if value:
        mr_boy=new_sort_boy_to_girl[key-1].index(value)
        mr_girl=new_sort_girl_to_boy[value-1].index(key)
        mr_boy_sum += mr_boy+1
        mr_girl_sum += mr_girl+1
amr_boy=mr_boy_sum/all_match_num
amr_girl=mr_girl_sum/all_match_num

print("amr_boy=",amr_boy)
print("amr_girl = ",amr_girl)
print("amr_total = ",amr_boy/2+amr_girl/2)
print("fe = ",amr_boy/2-amr_girl/2)
print("number of matched pair = ", all_match_num)
print("number of blocking pairs = ", all_match_num)

amr_boy= 2.9166666666666665
amr_girl =  6.416666666666667
amr_total =  4.666666666666667
fe =  -1.7500000000000002
number of matched pair =  12
number of blocking pairs =  12


In [135]:
# REPEATED Divorced matching until the end
# repeat for rep times
rep = 10
amr_girl=0
amr_boy=0
amr_boy_total=0
amr_girl_total=0
fe_repavg=0
blocking_pairs=0
matched_pairs=0

for i in range(1,rep):
    # select random divorce boys
    crossover= random.sample(range(1,size+1),12)
    # select pairs to divorce
    df2=df[df["boy_match"].isin(crossover)]
    # df2
    # divorced boy list
    boy_divorce=df2['boy_match'].values.tolist()
    boy_divorce.sort()
    boy_not_divorce=diff_of_two_list(boys,boy_divorce)
     # divorced girl list
    girl_divorce=df2['girl_match'].values.tolist()
    girl_divorce.sort()
    girl_not_divorce=diff_of_two_list(girls,girl_divorce)
    if __name__ == '__main__':
        sort_boy_to_girl_divorce = []
        for i in boy_divorce:
            item=diff_of_two_list(sort_boy_to_girl[i-1],girl_not_divorce)
            sort_boy_to_girl_divorce.append(item) 
    sort_boy_to_girl_divorce
    if __name__ == '__main__':
        sort_girl_to_boy_divorce = []
        for i in girl_divorce:
            item=diff_of_two_list(sort_girl_to_boy[i-1],boy_not_divorce)
            sort_girl_to_boy_divorce.append(item) 
            sort_girl_to_boy_divorce       
    result = find_free_partner(boy_divorce, girl_divorce, sort_boy_to_girl_divorce, sort_girl_to_boy_divorce) 
    new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy = tr_numbers(result,sort_boy_to_girl_divorce, sort_girl_to_boy_divorce)
    blocking_num,all_match_num = count_blocking_pairs(new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy)
    mr_boy_sum=0
    mr_girl_sum=0
    divorce_match=pd.DataFrame(list(divorce_match.items()),columns=['boy', 'girl'])
    girl_match=new_match_dict

    for key,value in girl_match.items():
        if value:
            mr_boy=new_sort_boy_to_girl[key-1].index(value)
            mr_girl=new_sort_girl_to_boy[value-1].index(key)
            mr_boy_sum += mr_boy+1
            mr_girl_sum += mr_girl+1

    amr_boy=mr_boy_sum/all_match_num
    amr_girl=mr_girl_sum/all_match_num
    print("amr_boy=",amr_boy)
    print("amr_girl = ",amr_girl)
    print("amr_total = ",amr_boy/2+amr_girl/2)
    print("fe = ",amr_boy/2-amr_girl/2)
    print("number of matched pair = ", all_match_num)
    print("number of blocking pairs = ", blocking_num)
    amr_boy_total+=amr_boy
    amr_girl_total+=amr_girl
    fe_repavg+=amr_boy/2-amr_girl/2
    blocking_pairs+=blocking_num
    matched_pairs+=all_match_num
    i+=1

print("amr_boy_repavg=",amr_boy_total/rep)
print("amr_girl_repavg=",amr_girl_total/rep)
print("amr_total_repavg=",amr_boy_total/2/rep+amr_girl_total/2/rep)
print("fe_repavg=",fe_repavg/rep)
print("matched_repavg=",matched_pairs/rep)
print("blocking_repavg=",blocking_pairs/rep)

amr_boy= 2.25
amr_girl =  8.333333333333334
amr_total =  5.291666666666667
fe =  -3.041666666666667
number of matched pair =  12
number of blocking pairs =  4
amr_boy= 2.3333333333333335
amr_girl =  7.75
amr_total =  5.041666666666667
fe =  -2.708333333333333
number of matched pair =  12
number of blocking pairs =  5
amr_boy= 2.0
amr_girl =  7.666666666666667
amr_total =  4.833333333333334
fe =  -2.8333333333333335
number of matched pair =  12
number of blocking pairs =  8
amr_boy= 2.0833333333333335
amr_girl =  5.75
amr_total =  3.916666666666667
fe =  -1.8333333333333333
number of matched pair =  12
number of blocking pairs =  4
amr_boy= 2.5
amr_girl =  6.0
amr_total =  4.25
fe =  -1.75
number of matched pair =  12
number of blocking pairs =  4
amr_boy= 2.8333333333333335
amr_girl =  6.416666666666667
amr_total =  4.625
fe =  -1.7916666666666667
number of matched pair =  12
number of blocking pairs =  1
amr_boy= 1.6666666666666667
amr_girl =  7.333333333333333
amr_total =  4.5
fe =  

In [120]:
def find_local_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy):
    
    #########################################
    #NOTE: When the number of boys are discrete with gaps
    #      turned into [1,2,3,... ] continuous number
    #########################################
    boy_index_dict = {}
    for i in range(len(boys)):
        boy_index_dict[boys[i]] = i+1
    girl_index_dict = {}
    for i in range(len(girls)):
        girl_index_dict[girls[i]] = i+1
    # old_boys = boys
    # old_girls = girls
    new_boys = list(range(1,len(boys)+1))
    new_girls = list(range(1,len(girls)+1))
    new_sort_boy_to_girl = []
    for items in sort_boy_to_girl:
        tmp = []
        for item in items:
            tmp.append(girl_index_dict[item])
        new_sort_boy_to_girl.append(tmp)
    new_sort_girl_to_boy = []
    for items in sort_girl_to_boy:
        tmp = []
        for item in items:
            tmp.append(boy_index_dict[item])
        new_sort_girl_to_boy.append(tmp)
    ###########################################################
    # print(girl_index_dict)
    # build dict：key: n-th boy，value: current girl choice，current_boys = {boys[0]:None, boys[1]:None, boys[2]:None, boys[3]:None ... }
    current_boys = dict(zip(new_boys, [0]*len(new_boys)))
    # build dict：key: n-th girl，value: current boy choice，current_girls = {girls[0]:None, girls[1]:None, girls[2]:None, boys[3]:None ... }
    current_girls = dict(zip(new_girls, [0]*len(new_girls)))
    # sumboys= sum(current_girls)
    count = len(boys)
 
 
    # build dict: next girl to choose for boys
    next_select = dict(zip(new_boys, [0]*len(new_boys)))
    # build sequence for every boy
    for i in range(count):
        # propose to the first girl
        temp = [new_girls[m-1] for m in new_sort_boy_to_girl[i]]
        # next time propose to the second
        next_select[new_boys[i]] = deque(temp)
 
 
    # dict of girl choose boy
    sort_girl = dict(zip(new_girls, [0]*len(new_boys)))
    for i in range(count):
        # girl rank boy
        temp = [[new_boys[m-1], 19-ind] for ind, m in enumerate(new_sort_girl_to_boy[i])]
        name = []
        match = []
        # for every girl
        for t in temp:
            # number of matched boy
            name.append(t[0])
            # rank of matched boy
            match.append(t[1])
        # get the ranking of girls
        sort_girl[new_girls[i]] = dict(zip(name, match))
 
 
    times = 0
    # while None in current_boys.values():    
    while times < 3:
        times += 1
        for i in range(count):
            # i-th boy proposes
            bid = new_boys[i]
            if current_boys[bid]:
                # skip if matched
                continue
            else:
                # 选choose the next on preference list
                select = next_select[bid][0]
                # if girl is unmatched
                if current_girls[select] == 0:
                    # match
                    current_boys[bid] = select
                    current_girls[select] = bid
                    next_select[bid].popleft()
                else:
                    # compare propose boy to the current match
                    # if the current match is preferred to the propose boy
                    # keep status quo 
                    if sort_girl[select][current_girls[select]] > sort_girl[select][bid]:
                        next_select[bid].popleft()
                    # if the reverse
                    # propose boy is preferred to the current match
                    # unmatch
                    # the girl match with propose boy
                    # current match can't propose again
                    else:
                        current_boys[current_girls[select]] = 0
                        current_boys[bid] = select
                        current_girls[select] = bid
                        next_select[bid].popleft()
    # print(current_boys,current_girls)
    # return to original index
    return_boys = dict()
    for key,value in current_boys.items():
        if value==0:
            return_boys[boys[key-1]] = 0
        else:
            return_boys[boys[key-1]] = girls[value-1]
    return return_boys


In [100]:
## initialization
size=20
boys = list(range(1,size+1))
girls = list(range(1,size+1))
# mr_girl_sum: sum of rankings of matched boy for girls
# mr_boy_sum: sum of rankings of matched girl for boys
mr_girl_sum=0
mr_boy_sum=0
# amr_girl: average of rankings of matched boy for girls (mr_girl_sum/rep)
# amr_boy:  average of rankings of matched girl for boys (mr_boy_sum/rep)
amr_girl=0
amr_boy=0
amr_boy_total=0
amr_girl_total=0
fe_repavg=0
sort_boy_to_girl= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
# print(sort_boy_to_girl)
dfboys = pd.DataFrame(pd.Series(sort_boy_to_girl),columns={'boy_preferece'})
dfboys.index=dfboys.index+1
dfboys=dfboys.reset_index().rename(columns = {'index':'boy'})
sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
# print(sort_girl_to_boy)
dfgirls = pd.DataFrame(pd.Series(sort_girl_to_boy),columns={'girl_preference'})
dfgirls.index=dfgirls.index+1
dfgirls=dfgirls.reset_index().rename(columns = {'index':'girl'})
# dfgirls
result = find_local_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy)
df = pd.DataFrame(pd.Series(result), columns=['girl_match'])
df=df.reset_index().rename(columns = {'index':'boy_match'})
#NOTE:test
new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy = tr_numbers(result,sort_boy_to_girl, sort_girl_to_boy)
blocking_num,all_match_num = count_blocking_pairs(new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy)
df
girl_match=new_match_dict
print(girl_match)
mr_boy_sum=0
mr_girl_sum=0
for key,value in girl_match.items():
    if value:
        mr_boy=new_sort_boy_to_girl[key-1].index(value)
        mr_girl=new_sort_girl_to_boy[value-1].index(key)
        mr_boy_sum += mr_boy+1
        mr_girl_sum += mr_girl+1

amr_boy=mr_boy_sum/all_match_num
amr_girl=mr_girl_sum/all_match_num
print("amr_boy=",amr_boy)
print("amr_girl = ",amr_girl)
print("amr_total = ",amr_boy/2+amr_girl/2)
print("fe = ",amr_boy/2-amr_girl/2)
print("number of matched pair = ", all_match_num)
print("number of blocking pairs = ", all_match_num)

{1: 1, 2: 17, 3: 4, 4: 6, 5: 13, 6: 12, 7: 3, 8: 10, 9: 14, 10: 9, 11: 0, 12: 18, 13: 8, 14: 7, 15: 11, 16: 0, 17: 19, 18: 20, 19: 5, 20: 0}
amr_boy= 1.4705882352941178
amr_girl =  10.0
amr_total =  5.735294117647059
fe =  -4.264705882352941
number of matched pair =  17
number of blocking pairs =  17


In [141]:
mr_girl_sum=0
mr_boy_sum=0
amr_girl=0
amr_boy=0
amr_boy_total=0
amr_girl_total=0
fe_repavg=0
matched_pairs=0
blocking_pairs=0
rep = 10 #add
for i in range(1,rep):
    # boys rank girls
    sort_boy_to_girl= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
    dfboys = pd.DataFrame(pd.Series(sort_boy_to_girl),columns={'boy_preferece'})
    # reindex from 1
    dfboys.index=dfboys.index+1
    dfboys=dfboys.reset_index().rename(columns = {'index':'boy'})        
    # girls rank boys
    sort_girl_to_boy= [random.sample(range(1,size+1),size) for _ in range(1, size+1)]
    # print boy rank boy list
    dfgirls = pd.DataFrame(pd.Series(sort_girl_to_boy),columns={'girl_preference'})
    # reindex
    dfgirls.index=dfgirls.index+1
    dfgirls=dfgirls.reset_index().rename(columns = {'index':'girl'})
    girl_match=pd.Series(find_free_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy))
    # 一次
    result = find_local_partner(boys, girls, sort_boy_to_girl, sort_girl_to_boy)
    df = pd.DataFrame(pd.Series(result), columns=['girl_match'])
    df=df.reset_index().rename(columns = {'index':'boy_match'})
    #NOTE:test
    new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy = tr_numbers(result,sort_boy_to_girl, sort_girl_to_boy)
    blocking_num,all_match_num = count_blocking_pairs(new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy)
    df
    girl_match=new_match_dict
    print(girl_match)
    mr_boy_sum=0
    mr_girl_sum=0
    for key,value in girl_match.items():
        if value:
            mr_boy=new_sort_boy_to_girl[key-1].index(value)
            mr_girl=new_sort_girl_to_boy[value-1].index(key)
            mr_boy_sum += mr_boy+1
            mr_girl_sum += mr_girl+1  
    amr_boy=mr_boy_sum/all_match_num
    amr_girl=mr_girl_sum/all_match_num
    print("amr_boy=",amr_boy)
    print("amr_girl=",amr_girl)
    print("amr_total=",amr_boy/2+amr_girl/2)
    print("fe=",amr_boy/2-amr_girl/2)
    print("number of matched pair = ", all_match_num)
    print("number of blocking pairs = ", all_match_num)
    amr_boy_total+=amr_boy
    amr_girl_total+=amr_girl
    fe_repavg+=amr_boy/2-amr_girl/2
    blocking_pairs+=blocking_num
    matched_pairs+=all_match_num
    i+=1
    
print("amr_boy_repavg=",amr_boy_total/rep)
print("amr_girl_repavg=",amr_girl_total/rep)
print("amr_total_repavg=",amr_boy_total/2/rep+amr_girl_total/2/rep)
print("fe_repavg=",fe_repavg/rep)
print("matched_repavg=",matched_pairs/rep)
print("blocking_repavg=",blocking_pairs/rep)

{1: 1, 2: 11, 3: 6, 4: 16, 5: 0, 6: 15, 7: 13, 8: 0, 9: 3, 10: 0, 11: 0, 12: 19, 13: 14, 14: 20, 15: 7, 16: 10, 17: 9, 18: 5, 19: 4, 20: 18}
amr_boy= 1.4375
amr_girl= 12.875
amr_total= 7.15625
fe= -5.71875
number of matched pair =  16
number of blocking pairs =  16
{1: 0, 2: 12, 3: 1, 4: 6, 5: 8, 6: 19, 7: 11, 8: 0, 9: 0, 10: 18, 11: 3, 12: 9, 13: 2, 14: 0, 15: 17, 16: 5, 17: 13, 18: 20, 19: 14, 20: 16}
amr_boy= 1.375
amr_girl= 9.9375
amr_total= 5.65625
fe= -4.28125
number of matched pair =  16
number of blocking pairs =  16
{1: 1, 2: 20, 3: 0, 4: 16, 5: 7, 6: 12, 7: 11, 8: 18, 9: 0, 10: 14, 11: 19, 12: 0, 13: 6, 14: 8, 15: 17, 16: 4, 17: 9, 18: 2, 19: 5, 20: 13}
amr_boy= 1.3529411764705883
amr_girl= 8.588235294117647
amr_total= 4.970588235294118
fe= -3.617647058823529
number of matched pair =  17
number of blocking pairs =  17
{1: 0, 2: 11, 3: 5, 4: 9, 5: 2, 6: 17, 7: 0, 8: 1, 9: 20, 10: 12, 11: 0, 12: 0, 13: 10, 14: 16, 15: 6, 16: 14, 17: 13, 18: 19, 19: 4, 20: 3}
amr_boy= 1.6875
amr

In [103]:
# Divorced people conduct Local Search for 3 times
prob=0.6
size=20
boys = list(range(1,size+1))
girls = list(range(1,size+1))
# mr_girl_sum: sum of rankings of matched boy for girls
# mr_boy_sum: sum of rankings of matched girl for boys
mr_girl_sum=0
mr_boy_sum=0
# amr_girl: average of rankings of matched boy for girls (mr_girl_sum/rep)
# amr_boy:  average of rankings of matched girl for boys (mr_boy_sum/rep)
amr_girl=0
amr_boy=0
amr_boy_total=0
amr_girl_total=0
fe_repavg=0
# repeat
# rep: repeated time
# random divorced pairs
df2=df[df["boy_match"].isin(crossover)]
# df2
# divorced boys list
boy_divorce=df2['boy_match'].values.tolist()
boy_divorce.sort()
# divorced girls list
girl_divorce=df2['girl_match'].values.tolist()
# boy_divorce
girl_divorce.sort()
# girl_divorce

# undivorced boys and girls (to be deleted from the list) 
girl_not_divorce=diff_of_two_list(girls,girl_divorce)
boy_not_divorce=diff_of_two_list(boys,boy_divorce)

# divorced boys rank divorced girls 
if __name__ == '__main__':
    sort_boy_to_girl_divorce = []
    for i in boy_divorce:
        # preference of boys for girls, MINUS undivorced girls
        item=diff_of_two_list(sort_boy_to_girl[i-1],girl_not_divorce)
        sort_boy_to_girl_divorce.append(item) 
# sort_boy_to_girl_divorce

# divorced girls rank divorced boys
if __name__ == '__main__':
    sort_girl_to_boy_divorce = []
    # for i in range(20):#NOTE：same
    for i in girl_divorce:
        # preference of girls for boys, MINUS undivorced boys
        item=diff_of_two_list(sort_girl_to_boy[i-1],boy_not_divorce)
        sort_girl_to_boy_divorce.append(item) 
# sort_girl_to_boy_divorce

result = find_local_partner(boy_divorce, girl_divorce, sort_boy_to_girl_divorce, sort_girl_to_boy_divorce) 
new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy = tr_numbers(result,sort_boy_to_girl_divorce, sort_girl_to_boy_divorce)
blocking_num,all_match_num = count_blocking_pairs(new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy)
divorce_match=pd.DataFrame(list(divorce_match.items()),columns=['boy', 'girl'])
girl_match=new_match_dict
for key,value in girl_match.items():
    if value:
        mr_boy=new_sort_boy_to_girl[key-1].index(value)
        mr_girl=new_sort_girl_to_boy[value-1].index(key)
        mr_boy_sum += mr_boy+1
        mr_girl_sum += mr_girl+1

    amr_boy=mr_boy_sum/all_match_num
    amr_girl=mr_girl_sum/all_match_num
    print("amr_boy=",amr_boy)
    print("amr_girl=",amr_girl)
    print("amr_total=",amr_boy/2+amr_girl/2)
    print("fe=",amr_boy/2-amr_girl/2)
    print("number of matched pair = ", all_match_num)
    print("number of blocking pairs = ", all_match_num)
    amr_boy_total+=amr_boy
    amr_girl_total+=amr_girl
    fe_repavg+=amr_boy/2-amr_girl/2
    blocking_pairs+=blocking_num
    matched_pairs+=all_match_num
    i+=1
    
print("amr_boy_repavg=",amr_boy_total/rep)
print("amr_girl_repavg=",amr_girl_total/rep)
print("amr_total_repavg=",amr_boy_total/2/rep+amr_girl_total/2/rep)
print("fe_repavg=",fe_repavg/rep)
print("matched_repavg=",matched_pairs/rep)
print("blocking_repavg=",blocking_pairs/rep)

amr_boy= 1.5
amr_girl =  6.75
amr_total =  4.125
fe =  -2.625
number of matched pair =  8
number of blocking pairs =  8


In [140]:
# REPEATED Divorced people conduct Local Search for 3 times
# mr_girl_sum: sum of rankings of matched boy for girls
# mr_boy_sum: sum of rankings of matched girl for boys
mr_girl_sum=0
mr_boy_sum=0
# amr_girl: average of rankings of matched boy for girls (mr_girl_sum/rep)
# amr_boy:  average of rankings of matched girl for boys (mr_boy_sum/rep)
amr_girl=0
amr_boy=0
amr_boy_total=0
amr_girl_total=0
fe_repavg=0
matched_pairs=0
blocking_pairs=0
# repeat for rep times
rep = 10 
for i in range(1,rep):
    # generate random divorced boys
    crossover= random.sample(range(1,size+1),12)
    # generate random divorced pairs
    df2=df[df["boy_match"].isin(crossover)]
    # df2
    # divorced boys list
    boy_divorce=df2['boy_match'].values.tolist()
    boy_divorce.sort()
    boy_not_divorce=diff_of_two_list(boys,boy_divorce)
     # divorced girls list
    girl_divorce=df2['girl_match'].values.tolist()
    girl_divorce.sort()
    girl_not_divorce=diff_of_two_list(girls,girl_divorce)

    # divorced boys rank divorced girls
    if __name__ == '__main__':
        sort_boy_to_girl_divorce = []
    # for i in range(20)
        for i in boy_divorce:
        # preference of girls for boys, MINUS undivorced girls
            item=diff_of_two_list(sort_boy_to_girl[i-1],girl_not_divorce)
            sort_boy_to_girl_divorce.append(item) 
    sort_boy_to_girl_divorce
    # divorced girls rank divorced boys
    if __name__ == '__main__':
        sort_girl_to_boy_divorce = []
    # for i in range(20):#NOTE：same
        for i in girl_divorce:
        # preference of girls for boys, MINUS undivorced girls
            item=diff_of_two_list(sort_girl_to_boy[i-1],boy_not_divorce)
            sort_girl_to_boy_divorce.append(item) 
            sort_girl_to_boy_divorce
    # 一次        
    result = find_local_partner(boy_divorce, girl_divorce, sort_boy_to_girl_divorce, sort_girl_to_boy_divorce) 
    new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy = tr_numbers(result,sort_boy_to_girl_divorce, sort_girl_to_boy_divorce)
    blocking_num,all_match_num = count_blocking_pairs(new_match_dict, new_sort_boy_to_girl, new_sort_girl_to_boy)
    divorce_match=pd.DataFrame(list(divorce_match.items()),columns=['boy', 'girl'])
    girl_match=new_match_dict

    for key,value in girl_match.items():
        if value:
            mr_boy=new_sort_boy_to_girl[key-1].index(value)
            mr_girl=new_sort_girl_to_boy[value-1].index(key)
            mr_boy_sum += mr_boy+1
            mr_girl_sum += mr_girl+1

    amr_boy=mr_boy_sum/all_match_num
    amr_girl=mr_girl_sum/all_match_num
    print("amr_boy=",amr_boy)
    print("amr_girl=",amr_girl)
    print("amr_total=",amr_boy/2+amr_girl/2)
    print("fe=",amr_boy/2-amr_girl/2)
    print("number of matched pair = ", all_match_num)
    print("number of blocking pairs = ", all_match_num)
    amr_boy_total+=amr_boy
    amr_girl_total+=amr_girl
    fe_repavg+=amr_boy/2-amr_girl/2
    blocking_pairs+=blocking_num
    matched_pairs+=all_match_num
    i+=1
    
print("amr_boy_repavg=",amr_boy_total/rep)
print("amr_girl_repavg=",amr_girl_total/rep)
print("amr_total_repavg=",amr_boy_total/2/rep+amr_girl_total/2/rep)
print("fe_repavg=",fe_repavg/rep)
print("matched_repavg=",matched_pairs/rep)
print("blocking_repavg=",blocking_pairs/rep)

amr_boy= 1.7777777777777777
amr_girl= 6.111111111111111
amr_total= 3.944444444444444
fe= -2.1666666666666665
number of matched pair =  9
number of blocking pairs =  9
amr_boy= 3.0
amr_girl= 11.444444444444445
amr_total= 7.222222222222222
fe= -4.222222222222222
number of matched pair =  9
number of blocking pairs =  9
amr_boy= 4.3
amr_girl= 15.6
amr_total= 9.95
fe= -5.65
number of matched pair =  10
number of blocking pairs =  10
amr_boy= 6.0
amr_girl= 20.7
amr_total= 13.35
fe= -7.35
number of matched pair =  10
number of blocking pairs =  10
amr_boy= 7.3
amr_girl= 27.5
amr_total= 17.4
fe= -10.1
number of matched pair =  10
number of blocking pairs =  10
amr_boy= 8.6
amr_girl= 34.7
amr_total= 21.650000000000002
fe= -13.05
number of matched pair =  10
number of blocking pairs =  10
amr_boy= 10.4
amr_girl= 38.8
amr_total= 24.599999999999998
fe= -14.2
number of matched pair =  10
number of blocking pairs =  10
amr_boy= 12.3
amr_girl= 44.9
amr_total= 28.6
fe= -16.299999999999997
number of m